# Tilman's No-Hangman Game

It features:

- IPyWidgets based UI
- Import of a Hangman word-bank from github
- Import and manipulation of an SVG clip-art graphics file to visualize game progress
- Scoring of word-difficulty by letter-frequency in English
- Random choice of word to guess by three levels of difficulty
- Multiple rounds
- Funky colored petals as a reward for winners

In [1]:
# Import library to load data from the web
# used for a list of words and graphics to visualize remaining lifes:
from urllib.request import urlopen

In [2]:
# Read a public hangman word-bank from github:
wordbank_url = 'https://gist.githubusercontent.com/alpha-tango/c3d2645817cf4af2aa45/raw/6c2b253372462aa9764240b15e48a785dd693148/Hangman_wordbank'
with urlopen(wordbank_url) as wordbank_file:
  wordbank = wordbank_file.read().decode('utf-8').split(', ')
wordbank = [ word.strip().lower() for word in wordbank ]
print(f'Successfully read {len(wordbank)} secret words.')

Successfully read 1022 secret words.


In [3]:
# Read a clip-art graphics of a flower with six petals which can represent the
# remaining lifes and parse the XML based SVG file as a tree such that it can be
# manipulated easily:
import xml.etree.ElementTree as ET
flower_url = 'https://openclipart.org/download/46117/Flower-6-red-petals-black-outline-green-leaf-01.svg'
with urlopen(flower_url) as flower_file:
  flower_tree = ET.parse(flower_file)

In [4]:
# Identify the SVG graphics elements that represent the six petals:
petal_path_ids = [
    element.attrib['id']
    for element in flower_tree.findall(".//{http://www.w3.org/2000/svg}path")
    if 'fill:#ff0000' in element.attrib.get('style') ]
print(f'Found {len(petal_path_ids)} SVG ids for petals.')

Found 6 SVG ids for petals.


In [5]:
# Define a function that randomly colors the six petals for a joyful reward for
# the winning player:
import copy
def funky_flower():
  funky_flower_tree = copy.deepcopy(flower_tree) # so we donot change the original
  for petal_id in petal_path_ids:
    petal_element = funky_flower_tree.find(f".//*[@id='{petal_id}']")
    petal_attrib = petal_element.attrib
    petal_attrib['visibility'] = 'visible'
    petal_attrib['style'] = petal_attrib['style'].replace(
          'fill:#ff0000',
          f'fill:hsl({random.randrange(240)+180} 95% 55%)')
  return ET.tostring(funky_flower_tree.getroot())

In [6]:
letter_frequencies = { # credit: https://gist.github.com/evilpacket/5973230
    "a": 8.167,
    "b": 1.492,
    "c": 2.782,
    "d": 4.253,
    "e": 12.702,
    "f": 2.228,
    "g": 2.015,
    "h": 6.094,
    "i": 6.966,
    "j": 0.153,
    "k": 0.772,
    "l": 4.025,
    "m": 2.406,
    "n": 6.749,
    "o": 7.507,
    "p": 1.929,
    "q": 0.095,
    "r": 5.987,
    "s": 6.327,
    "t": 9.056,
    "u": 2.758,
    "v": 0.978,
    "w": 2.360,
    "x": 0.150,
    "y": 1.974,
    "z": 0.074
}

sum(letter_frequencies.values())

99.999

In [7]:
import math
# Score a word by the frequency of its letters.
# This calculates the geometric mean of the individual letter frequencies in
# order to correlate with the chance to randomly pick said word but not to
# consider longer words as easier because in fact, they are less risky to guess.
# Then, it calculates the reciprocal to make less likely words more valuable.
def score_word(word: str):
  return 1/(math.prod([ letter_frequencies[_.lower()]/100 for _ in word ]))**(1/len(word))

# Score the words
word_scores = { word.lower(): score_word(word) for word in wordbank }
# Order the words by score
wordbank.sort(key = word_scores.__getitem__)
# Define levels
levels = [ 'easy', 'medium', 'hard' ]
# and partition the words into three equally-sized levels
words_per_level = len(wordbank) / len(levels) # intended to be float, so we donot miss any words when partitioning
words_by_level = {
    level: wordbank[round(i * words_per_level):round((i+1) * words_per_level)]
    for i, level in enumerate(levels)
}

word_game_score_min = 100
word_game_score_max = 10000
word_score_min = word_scores[wordbank[0]]
word_score_max = word_scores[wordbank[-1]]

def game_score_word(word: str):
  result = word_game_score_min + (word_game_score_max - word_game_score_min)*((word_scores[word.lower()]-word_score_min)/(word_score_max - word_score_min))
  granularity = 10**int(math.log(result, 10)) / 2
  rounded_result = int(round(result / granularity) * granularity)
  # print(result, granularity, rounded_result)
  return rounded_result

# { word: game_score_word(word) for word in wordbank }

In [8]:
# Main entry point, run this cell to start a new game after the previous cells
# have set up resources:

import ipywidgets as widgets

# Game configuration
miss_states = [
    # 0
    { "exclamation": "Alright, let's get some guesses going!" },
    # 1
    { "exclamation": "Oh well, one mistake, let's go on!" },
    # 2
    { "exclamation": "...fool me twice, shame on me!" },
    # 3
    { "exclamation": "Oh, oh, already used half of the mistakes! Take care!" },
    # 4
    { "exclamation": "Things are getting tight - only one mistake still allowed!" },
    # 5
    { "exclamation": "Think hard! You cannot be wrong anymore!" },
    # 6
    {
        "exclamation": "Oh no! I am sorry, you did not find the secret!",
        "lost": True
    }
]

# Functions used from multiple places in the game logic
def update_petals():
  for petal_id in petal_path_ids:
    flower_tree.find(f".//*[@id='{petal_id}']").attrib['visibility'] \
      = 'visible' if petal_id in petals_left else 'hidden'
  flower_image.value = ET.tostring(flower_tree.getroot())

def update_miss_state():
  update_petals()
  miss_state = miss_states[len(misses)]
  exclamation_label.value = miss_state['exclamation']
  return miss_state.get('lost', False)

def update_hidden_word():
  global hidden_word
  hidden_word = ''.join([ letter if letter in tries else ' ⎵ ' for letter in secret_word ])
  found_label.value = f'The secret word is: {hidden_word}'

# Set up the UI
level_select = widgets.Select(
    options=levels,
    description='Level:',
    rows=len(levels)
)
start_button = widgets.Button(description='Start game', icon='play')
score_text = widgets.Text(
    value='0',
    description='Score:',
    disabled=True
)
found_label = widgets.Label()
exclamation_label = widgets.Label()
guessed_label = widgets.Label()
guess_text = widgets.Text(
    value='',
    placeholder="What's your guess?",
    description='Next letter:',
    disabled=True
)
feedback_label = widgets.Label()

flower_image = widgets.Image(
    value = ET.tostring(flower_tree.getroot()),
    format = 'svg+xml',
    width = 150,
    height = 150)

game_layout = widgets.GridspecLayout(5, 4, width='80em')
game_layout[0, 0] = level_select
game_layout[0, 1:2] = start_button
game_layout[0, 2:] = score_text
game_layout[1:4, 0] = flower_image
game_layout[4, 0] = feedback_label
game_layout[1, 1:] = found_label
game_layout[2, 1:] = exclamation_label
game_layout[3, 1:] = guessed_label
game_layout[4, 1:] = guess_text

# Initialize game state
import random

def new_game():
  global level
  global secret_word
  global word_score
  global tries
  global misses
  global petals_left
  level = level_select.value
  secret_word = random.choice(words_by_level[level]).upper()
  word_score = game_score_word(secret_word)
  tries = []
  misses = []
  petals_left = petal_path_ids.copy()

  feedback_label.value = f'This word is worth {word_score}!'
  guessed_label.value = ''
  update_hidden_word()
  update_miss_state()
  level_select.disabled = True
  start_button.disabled = True
  guess_text.disabled = False
  guess_text.placeholder = "What's your guess?"

def end_game():
  level_select.disabled = False
  start_button.disabled = False
  guess_text.disabled = True

# Define and run the guess-feedback interaction
def check_guess(widget):
  global hidden_word
  guess = widget.value
  if len(guess) != 1:
    feedback_label.value = 'Please guess a single letter! Try again...'
    widget.value = ''
    return
  if not guess.isalpha():
    feedback_label.value = f'Please only guess letters, not {guess}! Try again...'
    widget.value = ''
    return
  guess = guess.upper()
  if tries.count(guess):
    feedback_label.value = f'You already guessed {guess}. Try again...'
    widget.value = ''
    return
  tries.append(guess)

  if guess in secret_word:
    feedback_label.value = 'Yay, that was a successful guess!'
    update_hidden_word()
  else:
    feedback_label.value = 'Sorry, that letter is not in the secret word!'
    misses.append(guess)
    petals_left.pop(random.randrange(len(petals_left)))

  guessed_label.value = f'You have already guessed: {" ".join(tries)}'
  widget.value = ''

  lost = update_miss_state()
  if lost:
    widget.placeholder = ">>> GAME OVER <<<"
    found_label.value = f'The secret word was: {secret_word} - you guessed: {hidden_word}'
    end_game()

  if hidden_word == secret_word:
    flower_image.value = funky_flower()
    exclamation_label.value = 'Congratulations! You found the secret word! ' \
    + ( 'You never missed! ' if not len(misses) else
        'You only missed once! ' if len(misses) == 1 else
       f'You missed {len(misses)} times. ' ) \
    + f'You scored {word_score} points!'
    widget.placeholder ="<<< WINNER! >>>"
    score_text.value = str(int(score_text.value) + word_score)
    end_game()


guess_text.on_submit(check_guess)
start_button.on_click(lambda widget: new_game())
display(game_layout)

GridspecLayout(children=(Select(description='Level:', layout=Layout(grid_area='widget001'), options=('easy', '…